In [18]:
import pandas as pd
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo

In [22]:
now = datetime.now(ZoneInfo('America/New_York'))
#now = datetime(2024, 11, 22, 18, 32, 34)
now

datetime.datetime(2024, 11, 22, 18, 35, 37, 483368, tzinfo=zoneinfo.ZoneInfo(key='America/New_York'))

In [32]:
games = pd.read_csv('data/games.csv')
games[games['Time'] <= now + timedelta(minutes = 30)]

TypeError: '<=' not supported between instances of 'str' and 'datetime.datetime'

In [29]:
now + timedelta(minutes = 30)

datetime.datetime(2024, 11, 22, 19, 5, 37, 483368, tzinfo=zoneinfo.ZoneInfo(key='America/New_York'))